# Main notebook file

This file is the main file refered to by the README.md file. Execute the code chunks from top to bottom. Additional code chunks which will be labeled as such.

### Loading in the functions (Package conformance checking)

In [ ]:
from conformance_checking import (
    throughput_time_to_xes,
    create_alignment,
    load_pnml, 
    make_dataframe_for_decision_tree,
    save_alignments,
    load_alignments,
    clean_alignments,
    generate_trace_encoding,
    make_alignments_table,
)


### Step 1: the computation step of the throughput time and converting it back to a XES format.
The first step undertaken was the computation step of the throughput time and converting it back to a XES format. The function  *throughput_time_to_xes* in the python file *xes_to_csv.py* is responsible for this step. 


In [2]:
throughput_time_to_xes("data/BPI2017Denied(3).xes","data/BPI2017Denied(3).csv","data/BPI2017Denied(3)_Throughput.xes")

c:\Users\Marijn\Documents\School\Process Mining\assignment\conformance_checking\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 3093/3093 [00:11<00:00, 269.24it/s]


Event log has been successfully exported to XES format!


### Step 2:  loading the normative process model (PetriNet).
The second step is loading the normative process model (PetriNet). The PetriNet given *BPI2017Denied_PetriNet.pmnl* is loaded in and visualized with the function *load_pnml* in the python file *alignment.py*. 

In [ ]:
PetriNet, iMarking, fMarking = load_pnml("data/BPI2017Denied_petriNet.pnml")

### Step 3: the generation of the alignments for each trace in the event log.
The third step undertaken was the generation of the alignments for each trace in the event log. After the eventlog and the normative process model are loaded the function *create_alignment.py* will create the optimal alignments with the event log and PetriNet as parameters. The alignments can also be saved as a .pkl file with the function *save_alignment* in the *alignment.py* python file. 

In [ ]:
alignments = create_alignment("data/BPI2017Denied(3)_Throughput.xes", PetriNet, iMarking, fMarking)

### Additional: Saving the alignments as a pickle file.

In [ ]:
save_alignments(alignments, 'data/alignments.pkl')
alignments = load_alignments('data/alignments.pkl')

### Cleaning step
Before applying a decision tree can be applied on the alignments, they have to be cleaned and encoded. The function clean_alignment in the python file *create_alignment.py* replaces the silent activity model move and log move *((none, >>) and (>>,none))* with a τ for additional readability and interpretability in the decision tree regressor model.

In [ ]:
clean_alignments = clean_alignments(alignments)

### Encoding step
The encoding step is undertaken with the functions *generate_trace_encoding* and *make_dataframe_for_decision_tree* both in the *create_alignment.py* python file. The *generate_trace_encoding* counts and aggregates all alignments on a trace level with one-hot encoding. *make_dataframe_for_decision_tree* adds the troughput time per trace and converts the object into a csv file for the decision tree. 

In [ ]:
move_count_per_trace = generate_trace_encoding(clean_alignments)
make_dataframe_for_decision_tree("data/BPI2017Denied(3)_Throughput.xes", move_count_per_trace, 'data/df_with_tau_for_decision_tree.csv')

## Additional: View alignment in table

In [ ]:
make_alignments_table(clean_alignments)

## Additional: Vizualize Both PetriNets

#### The given PetriNet (petrinet_graphs/given_petrinet)

![given_Petrinets](petrinet_graphs/given_petrinet.png)

#### The mined PetriNet (Based on the event log; (petrinet_graphs/mined_petrinet))

![mined_Petrinets](petrinet_graphs/mined_petrinet.png)

#### Step 4: The application of the set of encoded traces on a decision tree algorithm.
The fourth step is the application of the set of encoded traces on a decision tree algorithm. The decision tree regressor algorithm can be found in **NAME.ipynb**. Hyperparameter optimalization trough gridsearch cross validation is being utilized to enhance the overall performance of the decision tree regressor algorithm. The final hyperparameters can be found in a configuration file named: **config.json**.

### Step 5: Deriving Classification rules
Finally, the classification rules derived from the decision tree are produced with the function **NAME** in python file **NAME.py**. This function traverses through the derived decision tree and returns the classification rules and the amount of traces that apply for each classification rule in a csv. 